In [2]:
#take tags only from test5 database

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter


df = pd.read_csv('train.csv')

In [3]:
test_data = pd.read_csv('test5_all.csv')

In [4]:
# delete all data with empty tags from test data
test_data.dropna(subset=['photoTags'], inplace=True)

In [5]:
#take tags from the column 'photoTags'
tags = test_data.loc[:,'photoTags']

In [6]:
# join them to list
text = ' '.join(test_data['photoTags'])
text_list = text.split(' ')

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit_transform(tags)

<216987x185595 sparse matrix of type '<class 'numpy.int64'>'
	with 1762366 stored elements in Compressed Sparse Row format>

In [10]:
print(len(vectorizer.vocabulary_))

185595


In [23]:
# save the corpus into the file
import csv

dt = vectorizer.vocabulary_


d = pd.DataFrame.from_dict(dt, orient='index', dtype=None)

d.to_csv("corpus1.csv", encoding="utf-8", header=None)


In [24]:
#LOAD DATA
crp = pd.read_csv("corpus1.csv", header=None)
crp.to_csv("corpus1.csv", encoding="utf-8", header=['words','id'], index=None)

In [25]:
#LOAD DATA
crp = pd.read_csv("corpus1.csv")

#CLEAN DATA
crp['words'] = crp['words'].str.replace("\d+", '')
crp['words'] = crp['words'].str.replace("sec", '')
crp['words'] = crp['words'].str.replace("nabkv", '')

#train['words'] = train['words'].str.replace("^[0-9]*$", '')
#s = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", s)
crp['words'].replace('', np.nan, inplace=True)
crp = crp.dropna(subset=['words'])


In [26]:
crp = crp[crp['words'].str.len() > 3]


In [27]:
# save data
crp.to_csv("corpus.csv", encoding="utf-8",index=None)

In [28]:
# load data again 
crp = pd.read_csv('corpus.csv')
#take tags from the column 'photoTags'
tags = crp.loc[:,'words']

In [29]:
# so we have features 
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

tf_matrix = tfidf.fit_transform(tags)

In [30]:
vocab = tfidf.vocabulary_

In [32]:
# we have sparse matrix for 3 mln pictures 
# create KNN
from sklearn.neighbors import NearestNeighbors
train_data = pd.read_csv('dm_clean1.csv')
train_data_ = train_data.loc[:,'photoTags']
_vect = TfidfVectorizer(vocabulary = vocab)
train_c_matrix = _vect.fit_transform(train_data_)

nbrs = NearestNeighbors(n_neighbors = 2).fit(train_c_matrix)

In [33]:
test_data = pd.read_csv('test5_all.csv')
# delete all data with empty tags from test data
test_data.dropna(subset=['photoTags'], inplace=True)
test_data_ = test_data.loc[:,'photoTags']
new_vect = TfidfVectorizer(vocabulary = vocab)
test_c_matrix = new_vect.fit_transform(test_data_)


In [ ]:
data_lt = pd.DataFrame([], columns=['lt'])
data_ln = pd.DataFrame([], columns=['ln'])

for i in range(0,len(test_data)):
    distances, indices = nbrs.kneighbors(test_c_matrix[i])

#    print(indices)
    ind = indices[0][0]
#    if (distances[0][0]>1):
#            ind = indices[0][1]
#    print('ind',ind)
#    print(distances)
    lt =train_data.loc[ind,'latitude']
    lng = train_data.loc[ind,'longitude']

    df_lt = pd.DataFrame([lt], columns=['lt'])
    df_ln = pd.DataFrame([lng], columns=['ln'])
    
    data_lt = data_lt.append(df_lt)
    data_ln = data_ln.append(df_ln)
#    print('test:', test_data.loc[i,'photoTags'])
#    print('train:', train_data.loc[ind,'photoTags'])
#    print('-----------------------------')



In [57]:
from pandas import Series
test_data['lt'] = Series(data_lt, index=test_data.index)
test_data['ln'] = Series(data_ln, index=test_data.index)

ValueError: Wrong number of items passed 15, placement implies 216987